In [6]:
pip install transformers datasets torchaudio jiwer librosa evaluate

Note: you may need to restart the kernel to use updated packages.


In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# from datasets import load_dataset, DatasetDict

# dataset = DatasetDict()
# dataset["train"] = load_dataset("/content/drive/My Drive/dataset", split='train')  # Works in Colab

# print(dataset)

In [8]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()
dataset["train"] = load_dataset("itskavya/gp", split='train') # USE THIS ON YOUR MACHINE
print(dataset)

Resolving data files:   0%|          | 0/4932 [00:00<?, ?it/s]

10_13.mp3:   0%|          | 0.00/92.0k [00:00<?, ?B/s]

10_129.mp3:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

10_133.mp3:   0%|          | 0.00/65.0k [00:00<?, ?B/s]

10_128.mp3:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

...

9_chunk_9.wav:   0%|          | 0.00/4.94M [00:00<?, ?B/s]

9_chunk_5.wav:   0%|          | 0.00/4.94M [00:00<?, ?B/s]

train%2Fmetadata.csv:   0%|          | 0.00/560k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4931
    })
})


In [9]:
# Get column names to identify the second column
column_names = dataset["train"].column_names
second_column = column_names[1]  # Get the second column name
second_column

'transcription'

In [10]:
from datasets import Dataset

# Assuming your dataset is already loaded into 'my_dataset'
# Here's a way to filter out rows containing '[موسيقى]' in 'transcription'

def filter_dataset_1(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if '[موسيقى]' not in dataset['transcription'][i]:
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

# Applying the filter
filtered_dataset = filter_dataset_1(dataset['train'])

# Print the new dataset information to verify
print(filtered_dataset)

# If you want to update your original DatasetDict object
dataset['train'] = filtered_dataset

# Now 'my_dataset' will have the filtered dataset without rows containing '[موسيقى]'
print(dataset)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 4913
})
DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4913
    })
})


In [11]:
import re

# Remove only punctuation & numbers, but keep Arabic dialect words
chars_to_remove_regex =  '[\,\?\.\!\-\;\:\؟\"\“\%\‘\”\�\'\.\،]'

def clean_text(batch):
    batch["transcription"] = re.sub(chars_to_remove_regex, '', batch["transcription"]).strip()
    return batch

dataset = dataset.map(clean_text)

Map:   0%|          | 0/4913 [00:00<?, ? examples/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4913
    })
})

In [13]:
import re

# filter rows that have anything other than letters
def filter_dataset_2(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[^A-Za-z\u0621-\u064A\s]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset2 = filter_dataset_2(dataset['train'])

print(filtered_dataset2)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 26
})


In [14]:
for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")

C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\11_chunk_13.wav
عادل أص ثلاثة أيام كانت بالنسبة لي أسوأ ثلاثة أيام يوم ونصف مو أسوأ أيام الحمد لله الحمد لله على العافية يوم ونصف كانت سيئة جداً نفسياً عاد ما اقدر اولف مع أحد وكل اللي بالشقة يفهموني إلا واحد للي غبي أشرح له حاجه أقول له هي ايش تبغى مذري ايش ما يفهم الين ما  أكتبه له على الشاشة الين ما أوضح له تعبني فقعدت ياخوان يوم ونص مثل كذا الين جاب عمر فتح الباب دخل الشقة قال عادل تعال خل نروح مستشفى ثاني يمكن نروح عند دكتور ثاني أحسن وجيب علاجاتك معك قلت يالله تمام واخذ علاجاتي مئة واروح عند دكتور واخش عند دكتور استشاري


C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\12_chunk_1.wav
ويا أهلاً وسهلاً أنا ثامر وهذه الحلقة السابع عشر من بودكاست صح هذا البودكاست اللي أحكي لكم فيه عن جوانب مختلفة لتفاصيل ترهقنا الحياة فيها قبل ما نبدأ إذا ما كنت تعرف خدمة صح ببلص على أبل بودك

In [15]:
# filter rows that have both eng and arabic
def filter_dataset_3(dataset):
    filtered_rows = []
    for i in range(len(dataset['transcription'])):
        if bool(re.search(r'[A-Za-z]', dataset['transcription'][i]) and re.search(r'[\u0600-\u06FF]', dataset['transcription'][i])):
            filtered_rows.append(i)

    return dataset.select(filtered_rows)

filtered_dataset3 = filter_dataset_3(dataset['train'])

print(filtered_dataset3)

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 9
})


In [16]:
for i in range(len(filtered_dataset3)):
  print(filtered_dataset3[i]['audio']['path'])
  print(filtered_dataset3[i]['transcription'])
  print("\n")

C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\11_chunk_6.wav
متأكد ثلاث ايام لاتكلم و عطاني أربع علاجات مضاد و شراب مادري ايش   عاطاني كذا كم حاجه روح روح و لم قبل أن تبدأ المقطع أنت استراتيجي أنت عبقري أنت تحب البحار عادل جايب لك اليوم لعبة World of Warships أو عالم السفن الحربية اللعبة هذه يااخوان موجودة على البي سي و مجانية اللعبة عبارة عن معارك بحرية و السفن موجودة أكثر من خمسمئة سفينة موزعة على أحد عشر دولة و السفن هذا


C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\9_chunk_10.wav
الفيروس هذا اللي حملته اسمه no escape يعني ما فيه هرب سوي لها extraction ويشوفي شي صير تحذير للمرة الثانية سوي لها تشغيل على أي حال على طول ما عطانا حتى مقدمات ولا الأصوات المزينية ما فيه على طول حذر سوانا عد التشغيل ما كنا حذر


C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4

In [17]:
!pip install PyArabic

In [18]:
import pyarabic.araby as araby

# remove diacritic
def remove_diacritic(example):
    example['transcription'] = araby.strip_diacritics(example['transcription'])
    return example

dataset['train'] = dataset['train'].map(remove_diacritic)
print(dataset)

Map:   0%|          | 0/4913 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 4913
    })
})


In [19]:
filtered_dataset2 = filter_dataset_2(dataset['train'])

print(filtered_dataset2)

for i in range(len(filtered_dataset2)):
  print(filtered_dataset2[i]['audio']['path'])
  print(filtered_dataset2[i]['transcription'])
  print("\n")

Dataset({
    features: ['audio', 'transcription'],
    num_rows: 2
})
C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\6_210.mp3
يحفظكم ‏


C:\Users\Taliya Mira\.cache\huggingface\hub\datasets--itskavya--gp\snapshots\7df059ca039be1a675bdb4901b919930e4bc08d6\train\audio\9_chunk_4.wav
بس انا متخلف ما بشغله كل شي هنا قفلته اي صفر حماية بعدها الشي الثاني اللي سويته ان اي فيروس حصلتها على الانترنت حملتها وراح احملها باقي في فيروسات كثير ما حملتها راح احملها طبعا معظم الفيروسات بتجيك بصيغة متعارف عليها عند الناس يعني يحس بالامان هو يضغط عليها زي صيغة الصور PNG JPEG او ملفات فيديو زي MP4 DATAMOVE يعني تقريبا اي شي تحس المستخدم بيكون امن هو يفتح ايه




In [20]:
# split the dataset for testing
split_dataset = dataset["train"].train_test_split(test_size=0.2)
split_dataset

dataset['train'] = split_dataset['train']
dataset['test'] = split_dataset['test']
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3930
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 983
    })
})

In [21]:
def extract_all_chars(batch):
    all_texts = [" ".join(text) for text in batch["transcription"]]
    vocab_list = [list(set(text)) for text in all_texts]
    return {
        "vocab": vocab_list,
        "all_text": all_texts
    }

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 3930
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 983
    })
})

In [23]:
vocab_train = dataset['train'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset['train'].column_names)
vocab_test = dataset['test'].map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/3930 [00:00<?, ? examples/s]

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

In [24]:
# all distinct letters in the training dataset and test dataset
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [25]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 'ء': 1,
 'ا': 2,
 'ت': 3,
 'ح': 4,
 'خ': 5,
 'د': 6,
 'ر': 7,
 'ز': 8,
 'ش': 9,
 'ص': 10,
 'ط': 11,
 'ع': 12,
 'ق': 13,
 'ك': 14,
 'ل': 15,
 'م': 16,
 'ن': 17,
 'ه': 18,
 'و': 19,
 'ي': 20}

In [26]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [27]:
print(dataset['train'][0]['transcription'])

الوقت زاد الصداع والدوخه والحراره


In [28]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

23

In [29]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [30]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [31]:
repo_name = "wav2vec2-arabic-colab"

In [32]:
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\Taliya Mira\anaconda3\envs\torch\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Taliya Mira\.cache\huggingface\hub\models--Talyiamira--wav2vec2-arabic-colab. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/Talyiamira/wav2vec2-arabic-colab/commit/78c3d0f85dab891f706b7c6237844aaa6e79f247', commit_message='Upload tokenizer', commit_description='', oid='78c3d0f85dab891f706b7c6237844aaa6e79f247', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Talyiamira/wav2vec2-arabic-colab', endpoint='https://huggingface.co', repo_type='model', repo_id='Talyiamira/wav2vec2-arabic-colab'), pr_revision=None, pr_num=None)

In [33]:
input_str = filtered_dataset3[2]["transcription"]
labels = tokenizer(input_str).input_ids # it returns a dict of input ids and attention mask so just get the input ids
print(labels)
print(tokenizer.tokenize(input_str))
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

[11, 20, 21, 0, 21, 3, 4, 0, 15, 20, 0, 21, 21, 21, 21, 0, 21, 21, 21, 21, 21, 21, 21, 0, 5, 15, 19, 17, 20, 0, 21, 4, 2, 19, 15, 0, 21, 7, 21, 12, 0, 15, 15, 21, 21, 21, 21, 21, 21, 21, 21, 21, 0, 21, 21, 21, 21, 21, 21, 0, 21, 6, 19, 17, 0, 16, 2, 0, 21, 12, 11, 20, 18, 0, 21, 15, 19, 21, 0, 5, 15, 19, 17, 20, 0, 21, 9, 19, 21, 0, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 0, 16, 2, 0, 16, 17, 18, 0, 21, 2, 20, 6, 21, 0, 9, 21, 2, 15, 0, 14, 6, 18, 0, 21, 21, 0, 6, 18, 17, 20, 0, 13, 21, 15, 3, 18, 0, 16, 2, 0, 17, 21, 21, 21, 0, 17, 21, 15, 16, 18, 2, 0, 16, 7, 21, 0, 11, 20, 21, 0, 21, 17, 2, 0, 16, 2, 0, 4, 10, 15, 3, 0, 2, 15, 7, 2, 17, 21, 16, 19, 20, 7, 0, 5, 15, 19, 17, 20, 0, 21, 4, 2, 19, 15, 0, 21, 11, 21, 15, 0, 19, 20, 17, 6, 19, 8, 0, 19, 21, 9, 21, 15, 18, 2, 0, 16, 7, 21, 0, 21, 2, 17, 20, 21, 0, 19, 17, 9, 19, 21, 0, 2, 9, 20, 0, 20, 10, 20, 7, 0, 11, 20, 21, 0, 21, 19, 20, 17, 2, 0, 12, 2, 6, 21, 0, 2, 15, 3, 9, 21, 20, 15, 0, 17, 9, 19, 21, 0, 21, 21, 2, 0, 21, 2, 

In [34]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [35]:
#the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [36]:
#check the sampling rate
dataset['train'][0]["audio"]

{'path': 'C:\\Users\\Taliya Mira\\.cache\\huggingface\\hub\\datasets--itskavya--gp\\snapshots\\7df059ca039be1a675bdb4901b919930e4bc08d6\\train\\audio\\8_175.mp3',
 'array': array([ 7.15015531e-02,  1.20339438e-01,  1.71612978e-01, ...,
        -9.98508185e-05,  5.71432523e-04, -1.43049937e-03]),
 'sampling_rate': 44100}

In [37]:
# set the audio feature to the correct sampling rate

from datasets import Audio

dataset['train'] = dataset['train'].cast_column("audio", Audio(sampling_rate=16_000))
dataset['test'] = dataset['test'].cast_column("audio", Audio(sampling_rate=16_000))

In [38]:
dataset['train'][0]["audio"]

{'path': 'C:\\Users\\Taliya Mira\\.cache\\huggingface\\hub\\datasets--itskavya--gp\\snapshots\\7df059ca039be1a675bdb4901b919930e4bc08d6\\train\\audio\\8_175.mp3',
 'array': array([ 0.05898856,  0.21505   ,  0.27477747, ...,  0.00892218,
        -0.0012508 ,  0.00477683]),
 'sampling_rate': 16000}

In [39]:
#check that the data is correctly prepared

import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset['train'])-1)

print(dataset['train'][rand_int]["transcription"])
ipd.Audio(data=dataset['train'][rand_int]["audio"]["array"], autoplay=True, rate=16000)

التفكير جدا غلط كلام الناس ما ما احد


In [40]:
rand_int = random.randint(0, len(dataset['train'])-1)

print("Target text:", dataset['train'][rand_int]["transcription"])
print("Input array shape:", dataset['train'][rand_int]["audio"]["array"].shape)
print("Sampling rate:", dataset['train'][rand_int]["audio"]["sampling_rate"])

Target text: مثل أصابع اليد مو كلهم متساوين ومختلفين في تفكيرهم ونياتهم ومشاعرهم سواء كانت صادقة أو كاذبة فلذلك لازم نخالط الشخص اللي نبي نختاره ونجرب ونكتشف لأن نرتاح له ونعرفه أكثر وأكثر لأن التجارب هي اللي تعلمنا كيف نختار الصاحب والصديق سواء
Input array shape: (448000,)
Sampling rate: 16000


In [41]:
#apply the data preparation function to all examples
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [42]:
dataset['train'] = dataset['train'].map(prepare_dataset, remove_columns=dataset['train'].column_names)
dataset['test'] = dataset['test'].map(prepare_dataset, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/3930 [00:00<?, ? examples/s]

c:\Users\Taliya Mira\anaconda3\envs\torch\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/983 [00:00<?, ? examples/s]

In [43]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [44]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [45]:
import evaluate
metric = evaluate.load("wer")  # or any other metric

In [46]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [47]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
model.freeze_feature_extractor()

c:\Users\Taliya Mira\anaconda3\envs\torch\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:2176: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [49]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

c:\Users\Taliya Mira\anaconda3\envs\torch\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

C:\Users\Taliya Mira\AppData\Local\Temp\ipykernel_36576\381441507.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [51]:
trainer.train()

KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub()